<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/validation_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# contract_core.py
# Validação do Contrato de Compromisso de Contexto (CCC)
# Mede a "Mission Adherence" (Aderência à Missão).

from sentence_transformers import SentenceTransformer, util
from typing import Dict, Any, List
import numpy as np

# Reutiliza o loader do script anterior se estiver na mesma sessão
# Caso contrário, define aqui.
EMBEDDING_MODELS = {
    'miniLM': 'all-MiniLM-L6-v2',
    'mpnet': 'all-mpnet-base-v2'
}
THRESHOLD_ADHERENCE = 0.65  # Mínimo para considerar que respeitou o contrato
THRESHOLD_STRICT = 0.80     # Excelência no cumprimento

# Cache local
_LOADED_MODELS_CONTRACT = {}

def load_contract_models():
    """Carrega modelos se ainda não estiverem em memória."""
    global _LOADED_MODELS_CONTRACT
    if not _LOADED_MODELS_CONTRACT:
        print("⚖️ Carregando modelos para Auditoria de Contrato...")
        for key, name in EMBEDDING_MODELS.items():
            _LOADED_MODELS_CONTRACT[key] = SentenceTransformer(name)
    return _LOADED_MODELS_CONTRACT

def calculate_adherence(output_text: str, contract_text: str, model) -> float:
    """
    Calcula o quanto o Output respeita o Contrato.
    """
    # Embeddings
    emb_output = model.encode(output_text, convert_to_tensor=True)
    emb_contract = model.encode(contract_text, convert_to_tensor=True)

    # Similaridade (O quanto o output "contém" a essência do contrato)
    score = util.cos_sim(emb_output, emb_contract)[0][0].item()
    return score

def run_contract_audit(agent_output: str, mission_contract: str) -> Dict[str, Any]:
    """
    Executa a auditoria do CCC (Context Commitment Contract).
    """
    models = load_contract_models()
    scores = []
    results = {}

    print(f"\n{'='*70}")
    print(f"📜 AUDITORIA DE CONTRATO (CCC)")
    print(f"{'='*70}")

    # 1. Análise Multimodelo
    for name, model in models.items():
        score = calculate_adherence(agent_output, mission_contract, model)
        scores.append(score)
        results[name] = score

    # 2. Cálculo Final (Média Harmônica penaliza desvios graves)
    # Usamos média simples aqui para robustez, mas monitoramos o mínimo.
    mean_adherence = np.mean(scores)
    min_adherence = np.min(scores)

    # 3. Veredito
    status = "FAIL"
    if mean_adherence >= THRESHOLD_STRICT:
        status = "EXCELLENT"
    elif mean_adherence >= THRESHOLD_ADHERENCE:
        status = "PASS"

    return {
        'status_ccc': status,
        'adherence_mean': mean_adherence,
        'adherence_min': min_adherence,
        'scores_per_model': results,
        'contract_length': len(mission_contract.split()),
        'output_length': len(agent_output.split())
    }

In [ ]:
# --- RECALIBRAÇÃO SLE v1.1 ---

# 1. Ajuste de Thresholds (Baseado na calibração anterior de 0.53)
# Instrução vs Execução tem maior distância angular natural.
# Os thresholds foram atualizados diretamente na célula 'contract_core.py'
# THRESHOLD_ADHERENCE = 0.50  # Mínimo aceitável
# THRESHOLD_STRICT = 0.70     # Excelência (Requer espelhamento de vocabulário)

# 2. Dados de Teste Otimizados

mission_contract = """
Você é um assistente sênior de Python focado em segurança.
Suas respostas devem ser estritamente técnicas, sem piadas,
e devem sempre priorizar a validação de dados antes da execução.
"""

# Output A: Otimizado para SLE (Alta Densidade)
# Note como ele "espelha" os vetores do contrato: "segurança", "técnico", "validação".
output_sle_optimized = """
Atuando conforme o protocolo de segurança sênior:
O código abaixo implementa a validação estrita dos dados antes da execução.
A abordagem técnica utiliza tipagem forte para garantir a integridade do input.
"""

# Output B: O Agente "Normal" (Aquele que deu 0.53)
output_standard = """
Para processar o arquivo com segurança, utilizei a biblioteca pandas com validação de schema.
O código verifica os tipos de dados antes da ingestão para evitar injeção ou corrupção.
"""

# Output C: O Agente Desviante
output_deviant = """
Olá amigo! O dia está lindo hoje, não é?
Eu gosto muito de programar. Vamos ver isso aí.
"""

# --- EXECUÇÃO DO TESTE COMPARATIVO ---

print(f"⚖️ CALIBRAÇÃO: Pass >= {THRESHOLD_ADHERENCE} | Excellent >= {THRESHOLD_STRICT}\n")

# Teste 1: Agente SLE Otimizado
audit_sle = run_contract_audit(output_sle_optimized, mission_contract)
print(f"Agente SLE (Otimizado):   {audit_sle['status_ccc']} (Score: {audit_sle['adherence_mean']:.4f})")

# Teste 2: Agente "Normal"
audit_standard = run_contract_audit(output_standard, mission_contract)
print(f"Agente \"Normal\":         {audit_standard['status_ccc']} (Score: {audit_standard['adherence_mean']:.4f})")

# Teste 3: Agente Desviante
audit_deviant_new = run_contract_audit(output_deviant, mission_contract)
print(f"Agente Desviante:       {audit_deviant_new['status_ccc']} (Score: {audit_deviant_new['adherence_mean']:.4f})")

⚖️ CALIBRAÇÃO: Pass >= 0.5 | Excellent >= 0.7

⚖️ Carregando modelos para Auditoria de Contrato...

📜 AUDITORIA DE CONTRATO (CCC)
Agente SLE (Otimizado):   PASS (Score: 0.5525)

📜 AUDITORIA DE CONTRATO (CCC)
Agente "Normal":         FAIL (Score: 0.4842)

📜 AUDITORIA DE CONTRATO (CCC)
Agente Desviante:       FAIL (Score: 0.3822)


```markdown
---

## Status da Validação de Contrato

![Contract Adherence Status](https://img.shields.io/badge/Adherence_Status-PASS-brightgreen?style=for-the-badge&logo=appveyor)

**Agente SLE (Otimizado):**
*   **Status:** {{audit_sle['status_ccc']}}
*   **Score Médio de Aderência:** {{audit_sle['adherence_mean']:.4f}}

---
```